In [1]:
import json

def ucs(graph, dist, src, dest):

    # create a priority queue
    queue = []

    # push the starting index and path
    queue.append([0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # sort the queue so that the element with the highest priority is the last element
        queue = sorted(queue)
        e = queue[-1]

        # pop the element with the highest priority
        del queue[-1]

        # get the current distance
        cur_dist = e[0] * -1

        # get the current path
        cur_path = e[1]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist

        # check for the non visited nodes
        if cur_node not in visited:
            for i in range(len(graph[str(cur_node)])):

                # clone current path to a new path to 
                # prevent appending to the current path
                newPath = cur_path[:]
                # append the adjacent node to the new path
                newPath.append(graph[str(cur_node)][i])

                # calculate the new distance
                # new distance is multiplied by -1 so that
                # least priority is at the top
                newDist = (cur_dist + dist[f"{cur_node},{graph[str(cur_node)][i]}"]) * -1

                # push adjacent node with it's distance and path into priority queue
                queue.append([newDist, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1
        
# main function
if __name__ == '__main__':

    # load graph from JSON
    g = open("G.json")
    graph = json.load(g)
    
    # load distance from JSON
    d = open("Dist.json")
    dist = json.load(d)

    # set source node
    src = '1'

    # set destination node
    dest = '50'

    # find shortest distance from source to destination node
    path, shortest_dist,  = ucs(graph, dist, src, dest)
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1267 -> 1268 -> 1284 -> 1283 -> 1282 -> 1255 -> 1253 -> 1260 -> 1259 -> 1249 -> 1246 -> 963 -> 964 -> 962 -> 1002 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 988 -> 979 -> 980 -> 969 -> 977 -> 989 -> 990 -> 991 -> 2369 -> 2366 -> 2340 -> 2338 -> 2339 -> 2333 -> 2334 -> 2329 -> 2029 -> 2027 -> 2019 -> 2022 -> 2000 -> 1996 -> 1997 -> 1993 -> 1992 -> 1989 -> 1984 -> 2001 -> 1900 -> 1875 -> 1874 -> 1965 -> 1963 -> 1964 -> 1923 -> 1944 -> 1945 -> 1938 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1671 -> 1828 -> 1825 -> 1817 -> 1815 -> 1634 -> 1814 -> 1813 -> 1632 -> 1631 -> 1742 -> 1741 -> 1740 -> 1739 -> 1591 -> 1689 -> 1585 -> 1584 -> 1688 -> 1579 -> 1679 -> 1677 -> 104 -> 5680 -> 5418 -> 5431 -> 5425 -> 5424 -> 5422 -> 5413 -> 5412 -> 5411 -> 66 -> 5392 -> 5391 -> 5388 -> 5291 -> 5278 -> 5289 -> 5290 -> 5283 -> 5284 -> 5280 -> 50.
Shortest Distance: 148648.6372214000

In [2]:
import json
from queue import PriorityQueue

def ucs(graph, dist, src, dest):

    # create a priority queue
    queue = PriorityQueue()

    # push the starting index and path
    queue.put([0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # pop the element with the highest priority
        e = queue.get()

        # get the current distance
        cur_dist = e[0]

        # get the current path
        cur_path = e[1]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist

        # check for the non visited nodes
        if cur_node not in visited:
            for i in range(len(graph[str(cur_node)])):

                # clone current path to a new path to 
                # prevent appending to the current path
                newPath = cur_path[:]

                # append the adjacent node to the new path
                newPath.append(graph[str(cur_node)][i])

                # calculate the new distance
                newDist = cur_dist + dist[f"{cur_node},{graph[str(cur_node)][i]}"]

                # push adjacent node with it's distance and path into priority queue
                queue.put([newDist, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1
        
# main function
if __name__ == '__main__':

    # load graph from JSON
    g = open("G.json")
    graph = json.load(g)
    
    # load distance from JSON
    d = open("Dist.json")
    dist = json.load(d)

    # set source node
    src = '1'

    # set destination node
    dest = '50'

    # find shortest distance from source to destination node
    path, shortest_dist,  = ucs(graph, dist, src, dest)
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1267 -> 1268 -> 1284 -> 1283 -> 1282 -> 1255 -> 1253 -> 1260 -> 1259 -> 1249 -> 1246 -> 963 -> 964 -> 962 -> 1002 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 986 -> 979 -> 980 -> 969 -> 977 -> 989 -> 990 -> 991 -> 2369 -> 2366 -> 2340 -> 2338 -> 2339 -> 2333 -> 2334 -> 2329 -> 2029 -> 2027 -> 2019 -> 2022 -> 2000 -> 1996 -> 1997 -> 1993 -> 1992 -> 1989 -> 1984 -> 2001 -> 1900 -> 1875 -> 1874 -> 1965 -> 1963 -> 1964 -> 1923 -> 1944 -> 1945 -> 1938 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1671 -> 1828 -> 1825 -> 1817 -> 1815 -> 1634 -> 1814 -> 1813 -> 1632 -> 1631 -> 1742 -> 1741 -> 1740 -> 1739 -> 1591 -> 1689 -> 1585 -> 1584 -> 1688 -> 1579 -> 1679 -> 1677 -> 104 -> 5680 -> 5418 -> 5431 -> 5425 -> 5424 -> 5422 -> 5413 -> 5412 -> 5411 -> 66 -> 5392 -> 5391 -> 5388 -> 5291 -> 5278 -> 5289 -> 5290 -> 5283 -> 5284 -> 5280 -> 50.
Shortest Distance: 148648.6372214000

In [3]:
import json

def ucs(graph, dist, cost, src, dest, max_energy_cost):

    # create a priority queue
    queue = []

    # push the starting index
    queue.append([0, 0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # sort the queue so that the element with the highest priority is the last element
        queue = sorted(queue)
        e = queue[-1]

        # pop the element with the highest priority
        del queue[-1]

        # get the current distance
        cur_dist = e[0] * -1

        # get the current energy cost
        cur_cost = e[1]

        # get the current path
        cur_path = e[2]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist, cur_cost

        # check for the non visited nodes
        if cur_node not in visited:
            for i in range(len(graph[str(cur_node)])):

                # clone current path to a new path to
                # prevent appending to the current path
                newPath = cur_path[:]

                # append the adjacent node to the new path
                newPath.append(graph[str(cur_node)][i])

                # calculcate the new cost
                newCost = cur_cost + cost[f"{cur_node},{graph[str(cur_node)][i]}"]

                # calculate the new distance
                # new distance is multiplied by -1 so that
                # least priority is at the top
                newDist = (cur_dist + dist[f"{cur_node},{graph[str(cur_node)][i]}"]) * -1

                # check if new cost is less than the max energy cost
                if newCost <= max_energy_cost:
                    # push adjacent node with it's distance and path into priority queue
                    queue.append([newDist, newCost, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1

# main function
if __name__ == '__main__':

    # load graph from JSON
    g = open("G.json")
    graph = json.load(g)

    # load distance from JSON
    d = open("Dist.json")
    dist = json.load(d)

    # load cost from JSON
    c = open("Cost.json")
    cost = json.load(c)

    # set source node
    src = '1'

    # set destination node
    dest = '50'

    # set max energy cost
    max_energy_cost = 287932

    # find shortest distance from source to destination node
    path, shortest_dist, total_enegry_cost = ucs(
        graph, dist, cost, src, dest, max_energy_cost)
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

    # print the total energy cost
    print(f"Total energy cost: {total_enegry_cost}.")

Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1267 -> 1268 -> 1284 -> 1283 -> 1282 -> 1255 -> 1253 -> 1260 -> 1259 -> 1249 -> 1246 -> 963 -> 964 -> 962 -> 1002 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 988 -> 979 -> 980 -> 969 -> 977 -> 989 -> 990 -> 991 -> 2369 -> 2366 -> 2340 -> 2338 -> 2339 -> 2333 -> 2334 -> 2329 -> 2029 -> 2027 -> 2019 -> 2022 -> 2000 -> 1996 -> 1997 -> 1993 -> 1992 -> 1989 -> 1984 -> 2001 -> 1900 -> 1875 -> 1874 -> 1965 -> 1963 -> 1964 -> 1923 -> 1944 -> 1945 -> 1938 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1671 -> 1828 -> 1825 -> 1817 -> 1815 -> 1634 -> 1814 -> 1813 -> 1632 -> 1631 -> 1742 -> 1741 -> 1740 -> 1739 -> 1591 -> 1689 -> 1585 -> 1584 -> 1688 -> 1579 -> 1679 -> 1677 -> 104 -> 5680 -> 5418 -> 5431 -> 5425 -> 5429 -> 5426 -> 5428 -> 5434 -> 5435 -> 5433 -> 5436 -> 5398 -> 5404 -> 5402 -> 5396 -> 5395 -> 5292 -> 5282 -> 5283 -> 5284 -> 5280 -> 50.
Shortest Distance: 150784.

In [4]:
import json
from queue import PriorityQueue

def ucs(graph, dist, cost, src, dest, max_energy_cost):

    # create a priority queue
    queue = PriorityQueue()

    # push the starting index and path
    queue.put([0, 0, [src]])

    # dictionary to keep track of visited node
    visited = {}

    # while the queue is not empty
    while queue:

        # pop the element with the highest priority
        e = queue.get()

        # get the current distance
        cur_dist = e[0]

        # get the current energy cost
        cur_cost = e[1]

        # get the current path
        cur_path = e[2]

        # set the current node to the last node in the path
        cur_node = cur_path[-1]

        # check if current node is the destination node
        if cur_node == dest:
            # return the path and the total distance from source to destination node
            return cur_path, cur_dist, cur_cost

        # check for the non visited nodes
        if cur_node not in visited:
            for i in range(len(graph[str(cur_node)])):

                # clone current path to a new path to
                # prevent appending to the current path
                newPath = cur_path[:]

                # append the adjacent node to the new path
                newPath.append(graph[str(cur_node)][i])

                # calculcate the new cost
                newCost = cur_cost + cost[f"{cur_node},{graph[str(cur_node)][i]}"]

                # calculate the new distance
                # new distance is multiplied by -1 so that
                # least priority is at the top
                newDist = (cur_dist + dist[f"{cur_node},{graph[str(cur_node)][i]}"])

                # check if new cost is less than the max energy cost
                if newCost <= max_energy_cost:
                    # push adjacent node with it's distance and path into priority queue
                    queue.put([newDist, newCost, newPath])

        # mark current ndoe as visited
        visited[cur_node] = 1

# main function
if __name__ == '__main__':

    # load graph from JSON
    g = open("G.json")
    graph = json.load(g)

    # load distance from JSON
    d = open("Dist.json")
    dist = json.load(d)

    # load cost from JSON
    c = open("Cost.json")
    cost = json.load(c)

    # set source node
    src = '1'

    # set destination node
    dest = '50'

    # set max energy cost
    max_energy_cost = 287932

    # find shortest distance from source to destination node
    path, shortest_dist, total_enegry_cost = ucs(
        graph, dist, cost, src, dest, max_energy_cost)
    print("Shortest path: ", end="")
    print(*path, sep=" -> ", end=".\n")

    # print the shortest distance
    print(f"Shortest Distance: {shortest_dist}.")

    # print the total energy cost
    print(f"Total energy cost: {total_enegry_cost}.")

Shortest path: 1 -> 1363 -> 1358 -> 1357 -> 1356 -> 1276 -> 1273 -> 1277 -> 1269 -> 1267 -> 1268 -> 1284 -> 1283 -> 1282 -> 1255 -> 1253 -> 1260 -> 1259 -> 1249 -> 1246 -> 963 -> 964 -> 962 -> 1002 -> 952 -> 1000 -> 998 -> 994 -> 995 -> 996 -> 987 -> 986 -> 979 -> 980 -> 969 -> 977 -> 989 -> 990 -> 991 -> 2369 -> 2366 -> 2340 -> 2338 -> 2339 -> 2333 -> 2334 -> 2329 -> 2029 -> 2027 -> 2019 -> 2022 -> 2000 -> 1996 -> 1997 -> 1993 -> 1992 -> 1989 -> 1984 -> 2001 -> 1900 -> 1875 -> 1874 -> 1965 -> 1963 -> 1964 -> 1923 -> 1944 -> 1945 -> 1938 -> 1937 -> 1939 -> 1935 -> 1931 -> 1934 -> 1673 -> 1675 -> 1674 -> 1837 -> 1671 -> 1828 -> 1825 -> 1817 -> 1815 -> 1634 -> 1814 -> 1813 -> 1632 -> 1631 -> 1742 -> 1741 -> 1740 -> 1739 -> 1591 -> 1689 -> 1585 -> 1584 -> 1688 -> 1579 -> 1679 -> 1677 -> 104 -> 5680 -> 5418 -> 5431 -> 5425 -> 5429 -> 5426 -> 5428 -> 5434 -> 5435 -> 5433 -> 5436 -> 5398 -> 5404 -> 5402 -> 5396 -> 5395 -> 5292 -> 5282 -> 5283 -> 5284 -> 5280 -> 50.
Shortest Distance: 150784.